### 0. Import Libs

In [1]:
# kernel must be msd2
import sys;sys.path.append('../')
import os
import pandas as pd
import constant as c
import csv
import re
from util.epsg_transform import *
from shapely.geometry import *

### 1. Get Data

In [2]:
data_nm = 'popse'
dataset_nm = 'popse.csv'
data_path = os.path.join(c.SSD_PATH, data_nm, dataset_nm)
popse = pd.read_csv(data_path, quoting=csv.QUOTE_NONE, encoding = 'cp949')
popse.columns = c.POPSE_GEN_COLS
popse = popse.drop(columns = "NN")
popse = popse.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)
popse = popse[popse['단위'] == '명']
popse = popse[popse['구분별'] == '합계']
popse = popse.drop(columns=['구분별', '항목','단위'])
sggbjds = []
gu_nm = ''
for i in range(len(popse)):
    word = popse.iloc[i]['동별']
    if word[-1] == '구':
        gu_nm = word
        sggbjds.append(gu_nm)
    else:
        sggbjds.append(f'{gu_nm} {word}')
popse['구동명'] = sggbjds
int_cols = ['구동명', '202001', '202101', '202201']


In [3]:
popse

,동별,202001,202002,202003,202004,202005,202006,202007,202008,202009,...,202206,202207,202208,202209,202210,202211,202212,202301,202302,구동명
453,종로구,162106,162201,161984,161588,160701,160520,160166,159789,159842,...,153255,153246,153349,151999,152403,152537,152211,151998,152167,종로구
454,사직동,9840,9832,9841,9791,9761,9787,9803,9762,9848,...,9572,9547,9536,9379,9393,9373,9355,9389,9377,종로구 사직동
455,삼청동,3023,3009,2993,2981,2966,2973,2962,2956,2962,...,2692,2692,2702,2659,2651,2644,2642,2633,2626,종로구 삼청동
456,부암동,10542,10558,10597,10568,10467,10421,10404,10390,10347,...,9683,9676,9661,9590,9547,9569,9536,9491,9466,종로구 부암동
457,평창동,18806,18786,18768,18734,18690,18696,18635,18630,18618,...,18141,18148,18100,17952,17912,17856,17858,17828,17793,종로구 평창동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,암사1동,36689,36613,36492,36382,36291,36254,36242,36180,36060,...,34511,34455,34499,34305,34309,34301,34271,34286,34329,강동구 암사1동
902,천호2동,35843,35781,35714,35631,35640,35642,35609,35684,35688,...,33582,33607,33789,33813,33819,33844,33825,33872,33857,강동구 천호2동
903,길동,46230,46217,46063,46232,46731,46950,47085,47082,46997,...,45662,45616,45628,45420,45437,45395,45374,45299,45284,강동구 길동
904,상일1동,,,,,,,,,,...,39022,39077,39105,39038,39111,39133,39178,39308,39397,강동구 상일1동


In [ ]:
popse = popse[int_cols]

### 2. Get Coordinates

In [3]:
admdongxy_path = os.path.join(c.SSD_PATH, 'admdong_coord.xlsx')
admdongxy = pd.read_excel(admdongxy_path, sheet_name = '서울특별시') #, encoding = 'utf-8')
def remove_je(string):
    return re.sub(r'제(\d+)', r'\1', string)
admdongxy['읍면동/구'] = admdongxy['읍면동/구'].astype(str)
admdongxy['읍면동/구'] = admdongxy['읍면동/구'].apply(remove_je)
admdongxy = admdongxy[admdongxy['읍면동/구'] != 'nan']
admdongxy['구동명'] = admdongxy['시군구'] + ' ' + admdongxy['읍면동/구']
df_popse = pd.merge(popse, admdongxy, on = '구동명', how = 'left')
df_popse = df_popse[['시군구', '읍면동/구', '구동명', '202001', '202101', '202201', '위도', '경도']]
df_popse = df_popse.rename(columns={'위도': 'y', '경도': 'x', '시군구': 'sgg_nm', '읍면동/구': 'bjd_nm', '구동명': 'sggbjd_nm'})
df_popse = df_popse.dropna(subset=['bjd_nm'])
df_popse['x'] = pd.to_numeric(df_popse['x'], errors='coerce')
df_popse['y'] = pd.to_numeric(df_popse['y'], errors='coerce')
x_5179, y_5179 = coord_tranformation(list(df_popse.x), list(df_popse.y), "epsg:4326", "epsg:5179")
df_popse['x_5179'] = x_5179
df_popse['y_5179'] = y_5179
df_popse.to_csv('../asset/preprocess/features/df_popse.csv')

In [4]:
# Read df_popse
df_popse = pd.read_csv('../asset/preprocess/features/df_popse.csv', index_col = 0)